In [ ]:
# import the necessary packages
import numpy as np
from fetch.utils import get_model
from fetch.data_sequence import DataGenerator
import tensorflow as tf
import flask
import keras
import io

In [ ]:
# initialize our Flask application and the Keras model
app = flask.Flask(__name__)
graph = []
model = None
def load_model():
    global model
    global graph
    model = get_model('a')
    graph = tf.get_default_graph()

In [ ]:
@app.route("/predict", methods=["POST"])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    if flask.request.method == "POST":
        if flask.request.json:
            file = flask.request.json
            ft = np.array(file["data_freq_time"], dtype=np.float32)
            dt = np.array(file["data_dm_time"], dtype=np.float32)

            cand = {'data_freq_time': ft, 'data_dm_time': dt}
            with graph.as_default():
                preds = model.predict(cand)

            data["predictions"] = preds.tolist()
            data["success"] = True

    return flask.jsonify(data)

In [ ]:
# if this is the main thread of execution first load the model and
# then start the server
if __name__ == "__main__":
    print(("* Loading Keras model and Flask starting server..."
        "please wait until server has fully started"))
    load_model()
    app.run()